In [1]:
# import
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import datetime
from datetime import datetime
import re
import csv
import os
from io import BytesIO

# 인터파크
- 예외처리는 아직 안되어 있음.

In [1]:
tmp_list = [] # 합칠 리스트 생성

interpark_url = "https://tickets.interpark.com/contents/ranking?genre=musical"
r_i = requests.get(url=interpark_url)
html_i = r_i.content
soup_i = BeautifulSoup(html_i, 'html.parser')

# TOP3는 가장 위에 가로로 배열 -> tag_v
# TOP4~10은 그 아래라 세로로 배열 -> tag_h
tag_v = soup_i.findAll('div', class_="ranking-vertical-item_rankingItem__llUL_")
tag_h = soup_i.findAll('ul', class_='ranking-horizontal-item_rankingContents__z3wFG')

# 가로 TOP 3 리스트
for i in range(len(tag_v)):
    name_v = tag_v[i].find('li', class_='ranking-vertical-item_rankingGoodsName__m0gOz').text
    date_v =tag_v[i].find('div', class_='ranking-vertical-item_dateWrap__uZGMU').text
    location_v = tag_v[i].find('li', class_='ranking-vertical-item_placeName__4sHRS').text
    img_v = tag_v[i].find('div', class_='ranking-vertical-item_imageWrap__R6lkF').find('img')["src"]
    id_v = img_v.split('%')[8].split('_')[0].split('F')[1]
    goods_v = f"https://tickets.interpark.com/goods/{id_v}"
    img_url_v = f"https://tickets.interpark.com{img_v}"
    
    v_dict = {
        'name': name_v,
        'date' : date_v,
        'location' : location_v,
        'img_url' : img_url_v,
        'goods' : goods_v
    }
    tmp_list.append(v_dict)

# 세로 TOP 4~10 리스트
for j in range(7):
    name_h = tag_h[j].find('li', class_='ranking-horizontal-item_rankingTicketTitle__omJYh').text
    date_h = date_h = tag_h[j].find('div', class_='ranking-horizontal-item_dateWrap__tRsWh').text
    location_h = tag_h[j].find('li', class_='ranking-horizontal-item_placeName__zb9kN').text
    img_h = tag_h[j].find('div', class_='ranking-horizontal-item_imageWrap__owTl6').find('img')['src']
    id_h = img_h.split('%')[8].split('_')[0].split('F')[1]
    goods_h = f"https://tickets.interpark.com/goods/{id_h}"
    img_url_h = f"https://tickets.interpark.com{img_h}"
    
    h_dict = {
        'name': name_h,
        'date' : date_h,
        'location' : location_h,
        'img_url' : img_url_h,
        'goods' : goods_h
    }
    tmp_list.append(h_dict)

interpart_df = pd.DataFrame(tmp_list)
interpart_df

# rank, site_name 컬럼 추가 및 컬럼 순서 변경
interpart_df['rank'] = range(1, len(interpart_df) + 1)
interpart_df['site_name'] = '인터파크'
interpart_df = interpart_df[['site_name', 'rank', 'name', 'date', 'location', 'img_url', 'goods']]
interpart_df

NameError: name 'requests' is not defined

# 티켓링크
- 예외처리는 아직 안되어 있음.

In [11]:
url_t = "https://mapi.ticketlink.co.kr/mapi/ranking/genre/daily?categoryId=10&categoryId2=16&categoryId3=0&menu=RANKING"
r_t = requests.get(url=url_t)

# json 데이터 df 변경
data = json.loads(r_t.text)['data']['rankingList']
ticketlink_df = pd.DataFrame(data)

# 필요한 컬럼 추출 및 전처리
columns = ['productId', 'productName', 'startDate', 'endDate', 'hallName', 'imgUrl']
ticketlink_df = ticketlink_df[columns]

ticketlink_df['imgUrl'] = 'https:' + ticketlink_df['imgUrl']
ticketlink_df['productId'] = 'https://www.ticketlink.co.kr/product/' + ticketlink_df['productId'].astype(str)

ticketlink_df['startDate'] = ticketlink_df['startDate'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y.%m.%d.'))
ticketlink_df['endDate'] = ticketlink_df['endDate'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y.%m.%d.'))
ticketlink_df['date'] = ticketlink_df['startDate'] + ' ~ ' + ticketlink_df['endDate']
ticketlink_df.drop(columns=['startDate'], inplace=True)
ticketlink_df.drop(columns=['endDate'], inplace=True)

# 'rank', 'siteName' 컬럼 추가
ticketlink_df['rank'] = range(1, len(ticketlink_df) + 1)
ticketlink_df['siteName'] = '티켓링크'

# 컬럼 순서 변경
columns = ['siteName', 'rank', 'productName', 'date', 'hallName', 'imgUrl', 'productId']

ticketlink_df = ticketlink_df[columns]
ticketlink_df

,siteName,rank,productName,date,hallName,imgUrl,productId
0,티켓링크,1,뮤지컬 드라큘라 (Dracula：The Musical),2023.12.06. ~ 2024.03.03.,샤롯데씨어터,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46101
1,티켓링크,2,뮤지컬 <그날들> 10주년 기념 공연 - 고양,2023.12.22. ~ 2023.12.24.,고양아람누리 아람극장,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46949
2,티켓링크,3,뮤지컬 <문스토리>,2023.09.26. ~ 2023.12.10.,드림아트센터 4관,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/45428
3,티켓링크,4,뮤지컬 <타오르는 어둠 속에서>,2023.08.26. ~ 2023.11.26.,링크아트센터 페이코홀,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/44995
4,티켓링크,5,뮤지컬 일 테노레 (IL TENORE),2023.12.19. ~ 2024.02.25.,예술의전당 CJ토월극장,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46617
5,티켓링크,6,뮤지컬<후크>,2023.09.05. ~ 2023.11.26.,대학로 아트원씨어터 2관,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/45221
6,티켓링크,7,광주시립소년소녀합창단 제140회 정기공연 창작뮤지컬 <나를 노래해>,2023.12.01. ~ 2023.12.02.,빛고을 시민문화관,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46914
7,티켓링크,8,뮤지컬 <사의찬미> - 원주,2023.11.18. ~ 2023.11.18.,원주 치악예술관,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46559
8,티켓링크,9,뮤지컬 <스토리오브마이라이프>,2023.11.30. ~ 2024.02.18.,두산아트센터 연강홀,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46043
9,티켓링크,10,뮤지컬 <스모크>,2023.11.15. ~ 2024.02.04.,링크아트센터 벅스홀,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46233
